# For Capstone Project 
### Week 3 Final 
#### 1. Setting up the environment¶

In [17]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.40 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  32.88 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.16 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.53 MB/s
Libraries imported.


#### 2.Getting data from Wikipedia using BeautifulSoup

In [4]:
#https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source, 'lxml')  
table=soup.find('table')
table_rows=table.findAll('tr')
#l is the list to be passed through the function to obtain dataframe
l = []
for tr in table_rows:
    td = tr.find_all('td')
    raw_row = [tr.text for tr in td]
    #to check for empty rows and exclude these cases
    if raw_row == [] or raw_row[1] == 'Not assigned':
        continue

    #to parse neighborhood rows to remove \n
    edited_row=raw_row[0:len(raw_row)-1]
    edited_row.append(raw_row[-1][:-1])
    
    #check for empty neighborhoods to be replaced with borough 
    if edited_row[2] == 'Not assigned':
        edited_row[2] = edited_row[1]
    
    #adding entry to l
    l.append(edited_row)
    
    
df=pd.DataFrame(l, columns=["Postcode", "Borough", 'Neighborhood'])

df2=df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df2.head(15)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
df2.shape


(103, 3)

#### 3. Get geographical coordinates CSV files from "http://cocl.us/Geospatial_data"

In [6]:
geo_data=pd.read_csv("http://cocl.us/Geospatial_data")

geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 4.Combine two data frames

In [18]:
df_geo=geo_data.drop(columns=['Postal Code'])

df3=pd.concat([df2, df_geo], axis=1)
display(df3)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### 5.Clustering the neighborhoods in Toronto

In [32]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 6.use the Foursquare API to get the popular venues around each postcode.

Define Foursquare Credentials and Version

In [33]:
CLIENT_ID = 'C4J035AMEDZE3JXWQPOKLGTZTMUGRECHXNAJ1VFWUNLTC4G4' #  Foursquare ID
CLIENT_SECRET = 'XKJC5AOLMNPIEKZW51SVXEZOTZ1X5CNRMVNCREKDUL4NUNGH' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C4J035AMEDZE3JXWQPOKLGTZTMUGRECHXNAJ1VFWUNLTC4G4
CLIENT_SECRET:XKJC5AOLMNPIEKZW51SVXEZOTZ1X5CNRMVNCREKDUL4NUNGH


Define function to explore nearby venues in boroughs


In [34]:
LIMIT=100
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
toronto_venues = getNearbyVenues(names=df3['Borough'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

In [36]:
#check dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2243, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


check how many venues were returned for each borough

In [37]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1284,1284,1284,1284,1284,1284
East Toronto,123,123,123,123,123,123
East York,74,74,74,74,74,74
Etobicoke,69,69,69,69,69,69
Mississauga,11,11,11,11,11,11
North York,236,236,236,236,236,236
Queen's Park,42,42,42,42,42,42
Scarborough,91,91,91,91,91,91


In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


Analyze each borough

In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's St

In [42]:
toronto_onehot.shape

(2243, 280)

Group rows by borough and by taking the mean of the frequency of occurrence of each category

In [43]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's St

In [44]:
toronto_grouped.shape

(11, 280)

Print each borough along with the top 5 most common venues

In [45]:
num_top_venues = 5

for borough in toronto_grouped['Borough']:
    print("----"+borough+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == borough].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Café  0.05
3            Park  0.05
4     Pizza Place  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.04
3       Bakery  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.07
2      Ice Cream Shop  0.04
3  Italian Restaurant  0.04
4                Café  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Bank  0.04
2                 Park  0.04
3  Sporting Goods Shop  0.04
4        Grocery Store  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.12
1  Sandwich Place  0.07
2        Pharmacy  0.06
3     Coffee Shop  0.06
4            Park  0.04


----Mississauga----
                 venue  freq
0          Coffee Shop  0.18
1                Hotel  0.18
2       Sandwich Plac

function to sort venues in descending order

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create new dataframe and display top 10 venues for each borough

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Burger Joint,Clothing Store,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Gastropub
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Brewery,Sandwich Place,Bookstore,Pizza Place,American Restaurant
3,East York,Coffee Shop,Bank,Burger Joint,Sporting Goods Shop,Grocery Store,Pizza Place,Park,Pharmacy,Pet Store,Sandwich Place
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Fast Food Restaurant,Park,Gym,Grocery Store,Flower Shop,Convenience Store


Run k-means to cluster boroughs

In [48]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 4, 1, 0, 3, 4, 0], dtype=int32)

create new dataframe that includes the cluster as well as the top 10 venues for each borough

In [50]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park


Visualize the resulting clusters

In [53]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

Cluster 1

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
18,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
19,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
20,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
21,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
22,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
23,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
24,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
25,North York,2,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy
26,North York,1,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Japanese Restaurant,Café,Park,Grocery Store,Pizza Place,Pharmacy


Cluster 2

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Mississauga,3,1,Coffee Shop,Hotel,American Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,Sandwich Place,Dog Run


Cluster 3

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,York,1,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
74,York,1,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
80,York,3,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
81,York,3,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant
98,York,3,2,Park,Hockey Arena,Pharmacy,Sandwich Place,Field,Pizza Place,Convenience Store,Bakery,Trail,Turkish Restaurant


Cluster 4

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Queen's Park,0,3,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Bubble Tea Shop


Cluster 5

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
1,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
2,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
3,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
4,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
5,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
6,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
7,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
8,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park
9,Scarborough,2,4,Fast Food Restaurant,Bakery,Pizza Place,Breakfast Spot,Chinese Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Middle Eastern Restaurant,Park


#### Conclusion

After my analysis,I conclude that 5 clusters are:

1.Main Business District 
2.Country side 
3.Park and recreation facility area 
4.City Centre(Govement Buildings) 
5.Residential District 


1. First ordered list item
2. Another item
⋅⋅* Unordered sub-list. 
1. Actual numbers don't matter, just that it's a number
⋅⋅1. Ordered sub-list
4. And another item.

⋅⋅⋅You can have properly indented paragraphs within list items. Notice the blank line above, and the leading spaces (at least one, but we'll use three here to also align the raw Markdown).

⋅⋅⋅To have a line break without a paragraph, you will need to use two trailing spaces.⋅⋅
⋅⋅⋅Note that this line is separate, but within the same paragraph.⋅⋅
⋅⋅⋅(This is contrary to the typical GFM line break behaviour, where trailing spaces are not required.)

* Unordered list can use asterisks
- Or minuses
+ Or pluses